# Project 3

This project requires the implementation of an image classifier based on convolutional neural networks. The provided dataset (from [Lazebnik et al., 2006]),
contains 15 categories (office, kitchen, living room, bedroom, store, industrial,
tall building, inside city, street, highway, coast, open country, mountain, forest,
suburb), and is already divided in training set and test set.

### 1. train a shallow network

Libraries used.

In [ ]:
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Activation
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import os
import glob
import cv2 as cv2

In [ ]:
import utility as ut
from buildDataSet import retrieve_from, reshape

This first part is dedicated to data preprocessing.

In [ ]:
#path to the directories
PATH_TRAIN=r'C:\Users\eleon\Documents\Uni_Trieste\DSSC_II_year_I_semester\Computer Vision\Exam\train'
PATH_TEST=r'C:\Users\eleon\Documents\Uni_Trieste\DSSC_II_year_I_semester\Computer Vision\Exam\test'

#array of the class
LABELS=[os.path.basename(i) for i in glob.glob(PATH_TRAIN + '/*', recursive=True)]
NUM_CLASSES = len(LABELS) #15
print("Class number: ", NUM_CLASSES)
print("Class names: ", LABELS)

In [ ]:
#retrieve images from the directory and split the data set
X_train_raw, y_train_raw = retrieve_from(PATH_TRAIN, LABELS)
X_test, y_test = retrieve_from(PATH_TEST, LABELS)

print('Image Dimensions : ',X_test[0].shape)
print('len train x: ',len(X_train_raw))
print('len test x: ',len(X_test))
print('len train y: ',len(y_train_raw))
print('len test y: ',len(y_test))

Print a sample image to check the correctness of the procedure.

In [ ]:
cv2.imshow("Initial image", X_train_raw[9])
cv2.waitKey(0)
cv2.destroyAllWindows()
#note: remember to close the images to procede

Data manipulation.

In [ ]:
# Resize the images in a proper dimension
X_train_raw = reshape(X_train_raw, width = 64, height = 64, channel = 1)
X_test = reshape(X_test, width = 64, height = 64, channel = 1)

# Use One-Hot Encoding to convert the labels into a set of numbers to input into the neural network.
y_train_one_hot = to_categorical(y_train_raw)
y_test_one_hot = to_categorical(y_test)

# Split the training set between train 85% and validation set 15%
X_train, X_validation, y_train, y_validation = train_test_split(
    X_train_raw, y_train_one_hot, train_size=0.85, random_state=42)

print('Train set dimintion: ',len(X_train))
print('Validation set dimintion: ',len(X_validation))
print('Test set dimintion: ',len(X_test))

To build the model we need to create the given architecture using Sequential() and add the first layer, a convolution layer to extract features from the input image.

In [ ]:
# initial weights drawn from a Gaussian distribution with a mean of 0 and a standard deviation of 0.01
norm = ks.initializers.RandomNormal(mean=0.0, stddev=0.01, seed=None)

# employ Stochastic gradient descent with momentum optimization algorithm
sgd = ks.optimizers.SGD(momentum=0.9,nesterov=True)

base_model = Sequential([
    Conv2D(8, 3,strides=1, padding='same',input_shape=(64,64,1)),
    Activation('relu'),
    MaxPooling2D(pool_size=2,strides=2),
    Conv2D(16, 3,strides=1, padding='same'),
    Activation('relu'),
    MaxPooling2D(pool_size=2,strides=2),
    Conv2D(32, 3,strides=1, padding='same'),
    Activation('relu'),
    Flatten(),
    #set the initial bias values to 0
    Dense(NUM_CLASSES, activation='softmax',kernel_initializer=norm, bias_initializer='zeros')
])

base_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics = ['accuracy'])

Visualize training results, since we don't know the best number of epochs we compute a stopping critera .

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=5)
#use batch sized of 32
batch_sz = 32

base_history=base_model.fit(
    X_train,
    y_train,
    batch_size=batch_sz,
    epochs=20,
    validation_data=(X_validation,y_validation),
    shuffle=True,
    callbacks=[es]
)

Plot the model and the results

In [ ]:
ut.plot_acc_loss(base_history)
plt.savefig("accuracy and loss base model");

Print the accuracy and loss for the test set

In [ ]:
base_loss, base_accuracy = base_model.evaluate(X_test,y_test_one_hot)
print("loss: {:.2f}".format(base_loss))
print("accuracy: {:.2f}".format(base_accuracy))

Plot normalized confusion matrix

In [ ]:
y_pred_base = base_model.predict_classes(X_test)
ut.plot_confusion_matrix(y_test,y_pred_base, LABELS)
plt.savefig("confusion matrix base model");

In [ ]:
print('Classification Report')
print(ut.classification_report(y_test, y_pred_base, target_names=LABELS))
